In [2]:
import pandas as pd


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:

train_data = pd.read_csv("/content/drive/MyDrive/fraudTrain.csv")

sample_size_train = int(0.008 * len(train_data))

sample_data_train = train_data.sample(n=sample_size_train, random_state=42)

print("Sampled Training Data Shape:", sample_data_train.shape)
print("\nFirst Few Rows of Sampled Training Data:")
print(sample_data_train.head())

print(len(sample_data_train))
print(len(train_data))


Sampled Training Data Shape: (10373, 23)

First Few Rows of Sampled Training Data:
         Unnamed: 0 trans_date_trans_time            cc_num  \
1045211     1045211   2020-03-09 15:09:26      577588686219   
547406       547406   2019-08-22 15:49:01    30376238035123   
110142       110142   2019-03-04 01:34:16  4658490815480264   
1285953     1285953   2020-06-16 20:04:38  3514897282719543   
271705       271705   2019-05-14 05:54:48  6011381817520024   

                                            merchant        category     amt  \
1045211                              fraud_Towne LLC        misc_pos  194.51   
547406                             fraud_Friesen Ltd  health_fitness   52.32   
110142                                fraud_Mohr Inc    shopping_pos    6.53   
1285953                      fraud_Gaylord-Powlowski            home    7.33   
271705   fraud_Christiansen, Goyette and Schamberger   gas_transport   64.29   

           first        last gender                      

In [60]:
test_data = pd.read_csv("/content/drive/MyDrive/fraudTest.csv")
sample_size_test = int(0.008 * len(test_data))
sample_data_test = test_data.sample(n=sample_size_test, random_state=42)
print("Testing data shape:", sample_data_test.shape)
print("\nFirst few rows of testing data:")
print(sample_data_test.head())


Testing data shape: (4445, 23)

First few rows of testing data:
        Unnamed: 0 trans_date_trans_time           cc_num  \
119106      119106   2020-08-02 07:55:28  377895991033232   
179292      179292   2020-08-23 14:05:16   30364087349027   
540729      540729   2020-12-28 16:22:29   30328384440870   
374360      374360   2020-11-14 10:44:50   30364087349027   
314574      314574   2020-10-19 01:50:09    4198470814557   

                                    merchant       category    amt     first  \
119106   fraud_Bahringer, Schoen and Corkery   shopping_pos   1.07  Kimberly   
179292     fraud_Romaguera, Wehner and Tromp      kids_pets  94.99    Samuel   
540729                     fraud_Berge-Hills      kids_pets  31.28     Helen   
374360  fraud_Connelly, Reichert and Fritsch  gas_transport  73.06    Samuel   
314574                 fraud_Kuphal-Predovic       misc_net   9.99  Christie   

              last gender                      street  ...      lat     long  \
119106  

In [61]:
print("Missing values in training data:")
print(sample_data_train.isnull().sum())


Missing values in training data:
Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64


In [62]:
print("\nMissing values in testing data:")
print(sample_data_test.isnull().sum())


Missing values in testing data:
Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64


In [63]:

combined_data = pd.concat([sample_data_train, sample_data_test])

combined_data['trans_date_trans_time'] = pd.to_datetime(combined_data['trans_date_trans_time'])

combined_data['trans_year'] = combined_data['trans_date_trans_time'].dt.year
combined_data['trans_month'] = combined_data['trans_date_trans_time'].dt.month
combined_data['trans_day'] = combined_data['trans_date_trans_time'].dt.day
combined_data['trans_hour'] = combined_data['trans_date_trans_time'].dt.hour
combined_data['trans_minute'] = combined_data['trans_date_trans_time'].dt.minute

combined_data.drop(columns=['trans_date_trans_time'], inplace=True)

if 'dob' in combined_data.columns:
    combined_data['dob'] = pd.to_datetime(combined_data['dob'])
    combined_data['dob_year'] = combined_data['dob'].dt.year
    combined_data['dob_month'] = combined_data['dob'].dt.month
    combined_data['dob_day'] = combined_data['dob'].dt.day
    combined_data.drop(columns=['dob'], inplace=True)

combined_data_encoded = pd.get_dummies(combined_data, columns=['category', 'first', 'last', 'street', 'city', 'state', 'job', 'trans_num','gender','merchant'])

train_data_encoded = combined_data_encoded.iloc[:len(sample_data_train)]
test_data_encoded = combined_data_encoded.iloc[len(sample_data_train):]

print("First few rows of encoded training data:")
print(train_data_encoded.head())

print("\nFirst few rows of encoded testing data:")
print(test_data_encoded.head())


First few rows of encoded training data:
         Unnamed: 0            cc_num     amt    zip      lat      long  \
1045211     1045211      577588686219  194.51  15686  40.6153  -79.4545   
547406       547406    30376238035123   52.32  97476  42.8250 -124.4409   
110142       110142  4658490815480264    6.53  15449  39.9636  -79.7853   
1285953     1285953  3514897282719543    7.33  14425  42.9580  -77.3083   
271705       271705  6011381817520024   64.29  82221  41.6423 -104.1974   

         city_pop   unix_time  merch_lat  merch_long  ...  \
1045211       972  1362841766  40.420453  -78.865012  ...   
547406        217  1345650541  42.758860 -123.636337  ...   
110142        184  1330824856  40.475159  -78.898190  ...   
1285953     10717  1371413078  43.767506  -76.542384  ...   
271705        635  1336974888  41.040392 -104.092324  ...   

         merchant_fraud_Yost, Schamberger and Windler  \
1045211                                             0   
547406                     

In [71]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)


Shape of X_train: (317, 5064)
Shape of X_val: (212, 5064)
Shape of y_train: (317,)
Shape of y_val: (212,)


In [72]:
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier()
random_forest_model.fit(X_train, y_train)
train_predictions_rf = random_forest_model.predict(X_train)
val_predictions_rf = random_forest_model.predict(X_val)


In [73]:
from sklearn.metrics import accuracy_score, classification_report
print("Random Forest Training Performance:")
print("Accuracy:", accuracy_score(y_train, train_predictions_rf))
print("Classification Report:")
print(classification_report(y_train, train_predictions_rf))


Random Forest Training Performance:
Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       316
           1       1.00      1.00      1.00         1

    accuracy                           1.00       317
   macro avg       1.00      1.00      1.00       317
weighted avg       1.00      1.00      1.00       317

